# **Finding Lane Lines on the Road** 

### REFLECTION!

This is actually my *first* attempt at drawing lane lines. Here's basically how it works:
1. Get image data, turn it into grayscale, draw a region of interest and then do cannied and hough transforms on it to finally get a list of lines called *houghlines*
2. Now, I look through the houghlines and separate them into two new lists, *left* and *right*. If a line's slope is -ve, I append it into 'left' and if it is +ve, I put it into 'right'. Flat lines are therefore ignored.
3. For each lane side, I use a function called 'find_endpts' which finds 2 points: first, the 'lowest' point on the lane which is closest to the camera, and the 'highest' point on the lane which is farthest away. I will use these two points to draw a line between them.
4. I use a function called 'lane_line' to return a new line which has the slope and intercept of the end-points (found in step 3), and then draws a complete lane marking.

I actually got done with this 2 days ago, but have been trying to improve on it because this algorithm breaks into a million little pieces when I try to use it on the yellow and challenge videos. :(

The reason is simple - if a particular line has an infinite slope (which happens to be the case in both those videos). My real code has actually come a loooong way in these 2 days and uses a totally different algorithm (and believe me I've been dreaming algorithms in my sleep trying to fix this), but I just HAD to submit this on the deadline. So here it goes.... *(cries softly)*

In [5]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
%matplotlib inline

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [6]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=3):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

# Didn't end up this function in my final algorithm
def find_endpts(lines):
    lowest = [lines[0][0][0], lines[0][0][1]]
    highest = [lines[0][0][2], lines[0][0][3]]
    
    for line in lines:
        if line[0][1] > lowest[1]:
            lowest = [line[0][0], line[0][1]]
        if line[0][3] < highest[1]:
            highest = [line[0][2], line[0][3]]
    return [lowest, highest]

# Didn't end up this function in my final algorithm
def lane_line(x1, y1, x2, y2):
    m = (y2-y1)/(x2-x1)
    b = (y2 - (m*x2))
    
    v1 = 539
    v2 = 320
    
    u1 = int((v1-b)/m)
    u2 = int((v2-b)/m)
    
    return [u1, v1, u2, v2]


def valid_slope(ends):
    if (ends[0][0] == ends[1][0]) or (ends[0][1] == ends[1][1]):
        return False
    else:
        return True
    
def get_slope(lst):
    return (((lst[3]-lst[1])/(lst[2]-lst[0]))*180/np.pi)

def median_slope(lines):
    slopes = []
    for line in lines:
        slopes.append(((line[3]-line[1])/(line[2]-line[0]))*180/np.pi)
    return np.median(slopes)


## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [15]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [14]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    
    global left_previous
    global right_previous
    
    width = image.shape[1]
    height = image.shape[0]
    center_height = int(height * 0.5926)
    left_bottom = int(width * 0.10625)
    right_bottom = int(width - (width * 0.0625))
    left_center = int((width/2) - (width * 0.0108))
    right_center = int((width/2) + (width * 0.0521))
    
    blue = image[:,:,0]
    blurred = gaussian_blur(blue, 7)
    cannied = canny(blurred, 80, 200)
    vertices = np.array([[[left_bottom, height], [left_center, center_height], [right_center, center_height], [right_bottom, height]]])
    cannied_region = region_of_interest(cannied, vertices)

    line_img = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    #cv2.fillPoly(line_img, vertices, [100,0,0])
    houghlines = cv2.HoughLinesP(cannied_region, 0.5, np.pi/360, 4, 30, 5)

    houghlines = np.squeeze(houghlines)

    left = []
    right = []

    if len(houghlines) > 1:
        for line in houghlines:
            if (line[3]-line[1])/(line[2]-line[0]) < 0:
                left.append(line)
                #cv2.line(line_img, (line[0], line[1]), (line[2], line[3]), [0,0,200], 8)
            elif (line[3]-line[1])/(line[2]-line[0]) > 0:
                right.append(line)
                #cv2.line(line_img, (line[0], line[1]), (line[2], line[3]), [0,0,200], 8)

    slope_left = median_slope(left)
    slope_right = median_slope(right)

    new_left = []
    new_right = []

    for line in left:
        if (abs(get_slope(line) - slope_left) < 8):
            new_left.append(line)

    for line in right:
        if (abs(get_slope(line) - slope_right) < 8):
            new_right.append(line)

    # Now lets use cv2.fitline

    left_points = []
    for line in new_left:
        left_points.append((line[0], line[1]))
        left_points.append((line[2], line[3]))

    right_points = []
    for line in new_right:
        right_points.append((line[0], line[1]))
        right_points.append((line[2], line[3]))


    [vxl, vyl, xl, yl] = cv2.fitLine(np.array(left_points, dtype=np.int32), cv2.DIST_L2, 0, 0.01, 0.01)
    [vxr, vyr, xr, yr] = cv2.fitLine(np.array(right_points, dtype=np.int32), cv2.DIST_L2, 0, 0.01, 0.01)

    #Get current slopes
    ml = vyl/vxl
    mr = vyr/vxr

    #Compare with previous frame
    if len(left_previous) == 2:
        [m_old, b_old] = left_previous
        if (abs((ml*180/np.pi) - m_old) > 10):
            ml = (m_old * np.pi/180)*0.8 + ml*0.2
        if (abs((ml*180/np.pi) - m_old) > 5):
            ml = (m_old * np.pi/180)*0.7 + ml*0.3
        else:
            ml = (m_old * np.pi/180)*0.5 + ml*0.5

    if len(right_previous) == 2:
        [m_old, b_old] = right_previous
        if (abs((mr*180/np.pi) - m_old) > 10):
            mr = (m_old * np.pi/180)*0.8 + mr*0.2
        if (abs((mr*180/np.pi) - m_old) > 5):
            mr = (m_old * np.pi/180)*0.7 + mr*0.3
        else:
            mr = (m_old * np.pi/180)*0.5 + mr*0.5

    # Get current intercept using new slopes
    bl = (yl - ml*xl)
    br = (yr - mr*xr)

    # Set new REFERENCE
    left_previous = [ml*180/np.pi, bl]
    right_previous = [mr*180/np.pi, br]

    #Draw the lines
    x1l = int((height - bl)/ml)
    x2l = int((center_height - bl)/ml)
    cv2.line(line_img, (x1l, height), (x2l, center_height), [255,0,0], 5)

    x1r = int((height - br)/mr)
    x2r = int((center_height - br)/mr)
    cv2.line(line_img, (x1r, height), (x2r, center_height), [255,0,0], 5)
    
    overlay = weighted_img(line_img, image)
    return overlay

Let's try the one with the solid white lane on the right first ...

In [16]:
white_output = 'white2.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from moviepy.video.VideoClip import VideoClip
from IPython.display import HTML

global width
global height
global center_height
global left_bottom
global right_bottom
global left_previous
global right_previous
left_previous = []
right_previous = []

white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white2.mp4
[MoviePy] Writing video white2.mp4


100%|█████████▉| 221/222 [00:07<00:00, 28.13it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white2.mp4 

CPU times: user 3.86 s, sys: 1.19 s, total: 5.05 s
Wall time: 8.09 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [17]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Well, well, well... I had to try over 4 different algorithms before finally getting this output. My first code produced fine lane markings but they were a little jittery. And in the end, simply removing the jitter took me 2 whole days.

The code works okay-ish, but it breaks and throws and error when I try it on the challenge video below. Apparently, that happens when the program doesn't find *any* lines on a particular lane side, and the `cv2.fitLine` function doesn't work on empty points arrays. I tried to fix that but quite honestly I'm at the end of my patience here, haha.

It can definitely be improved, and other students have tried using all sorts of filters from Kalman to Ransac. I think that's pretty advanced for me right now so I'll just settle for this non-jittery output (*I guess the challenge video will remain a mystery for me, unless YOU, my dear gentle reader, can throw some light on this issue and fix my code!*)


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))